In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from xiao_utils import f

In [2]:
# 将level_id字段中的-替换为np.nan
df = pd.read_csv('../../data/origin/[new] yancheng_train_20171226.csv', dtype={'sale_date':str}, na_values=['-'], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'], format='%Y%m')

# 将price_level字段转换成有序类别的类型，并用其数值填入该列。
df['price_level'] = df['price_level'].astype('category', categories=['5WL','5-8W','8-10W','10-15W','15-20W','20-25W','25-35W','35-50W','50-75W'], ordered=True)
df['price_level'] = df['price_level'].cat.codes

# 待选方案：先把power和扭矩字段带/的行复制一份，然后将新行里的销量清零，将原行和新行的power和扭矩字段的值分别赋为slash前后的值。
# 现行方案：先他娘的直接把slash和后面的值删掉。省得影响记录条数相关的统计量。
def process_power_and_torque(s):
    return s.split('/')[0]
df['power'] = df['power'].astype(str).apply(process_power_and_torque).astype(float) #[18600]
df['engine_torque'] = df['engine_torque'].astype(str).apply(process_power_and_torque).astype(float)

# # -------------------------------------------------------------
# # 把2017年11月的数据拼进来，一块填入其特征，用于最终输出要提交的结果。
# empty_Nov = pd.read_csv('../../data/origin/yancheng_testA_20171225.csv', dtype={'predict_date':str}, na_values=['-'], low_memory=False)
# empty_Nov['predict_date']= pd.to_datetime(empty_Nov['predict_date'], format='%Y%m')
# empty_Nov.rename(columns = {'predict_date': 'sale_date', 'predict_quantity':'sale_quantity'}, inplace = True)


# # 读取玩了，先不急着拼，先把车型到品牌的映射关系join进来
# class_to_brand = df[['class_id','brand_id']].groupby(['class_id']).mean().reset_index()
# empyt_Nov = pd.merge(left=empty_Nov, right=class_to_brand, on='class_id', how='left')
# empty_Nov['brand_id']= class_to_brand['brand_id']
# # empty_Nov
# # class_to_brand

# # class_to_brand

# # 读取完了，拼上去
# df = pd.concat([df, empty_Nov])
# df.info()

# ------------------------------------------------------------------
df

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,...,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,2016-09-01,289403,94,12,2,1,1.0,1,6,MT,...,170.0,4440,1833,1545,1695,1320,5,2700,1556,1562
1,2016-09-01,745137,435,637,3,2,1.0,2,6,DCT,...,159.0,4534,1823,1483,1711,1336,5,2648,1553,1544
2,2016-09-01,714860,180,831,3,2,2.0,3,6,AT,...,176.0,4720,1815,1465,1860,1459,5,2770,1579,1589
3,2016-09-01,175962,40,750,3,2,1.0,4,6,AT,...,155.0,4475,1706,1469,1625,1145,5,2603,1460,1500
4,2016-09-01,270690,19,98,2,3,3.0,1,5,MT,...,146.5,4415,1685,1850,1825,1236,5,2720,1420,1440
5,2016-09-01,290854,53,537,2,1,1.0,5,0,CVT,...,190.0,4585,1820,1685,2010,1540,5,2620,1580,1580
6,2016-09-01,692703,81,807,2,1,1.0,1,5,MT,...,202.0,4510,1852,1677,1900,1450,5,2650,1575,1570
7,2016-09-01,978089,37,12,2,2,1.0,1,5,MT,...,140.0,4425,1789,1470,1611,1236,5,2650,1502,1492
8,2016-09-01,219195,115,638,2,1,3.0,1,6,AT,...,210.0,4335,1814,1695,1895,1520,5,2560,1525,1520
9,2016-09-01,851857,78,75,2,1,2.0,2,8,AT,...,400.0,4812,1903,1680,2375,1885,5,2857,1645,1645


In [4]:
gs = df.groupby(['class_id','sale_date'])['sale_quantity'].sum()


class_id  sale_date 
103507    2015-03-01      58
          2015-04-01     232
          2015-05-01     226
          2015-06-01     286
          2015-07-01     297
          2015-08-01     355
          2015-09-01     442
          2015-10-01    1050
          2015-11-01     481
          2015-12-01    1433
          2016-01-01     993
          2016-02-01     953
          2016-03-01     650
          2016-04-01     145
          2016-05-01     772
          2016-06-01     829
          2016-07-01     378
          2016-08-01     406
          2016-09-01     684
          2016-10-01     632
          2016-11-01     876
          2016-12-01    1653
          2017-01-01     499
          2017-02-01     435
          2017-03-01     371
          2017-04-01     290
          2017-05-01     429
          2017-06-01     371
          2017-07-01     313
          2017-08-01     180
                        ... 
978089    2015-05-01     418
          2015-06-01     372
          2015-07-01  

In [45]:
# 统计每个月有多少个车型在售
selling_cls_in_month = df[['sale_date', 'class_id']].groupby(['sale_date', 'class_id']).size().unstack()
selling_cls_in_month.count()

class_id
103507    32
124140    42
125403    70
136916    70
169673     5
175962    59
178529    70
186250    21
194201     2
194450    69
198427    53
206765    28
209945    70
219195    39
221795    55
245609    19
248352    70
249875    12
250658    20
265980    69
270690    34
281301    70
281792    41
289386    41
289403    18
290854    70
291086    70
291514    60
302513    12
304458    70
          ..
745137    66
750340    28
760412    10
786351    62
789290     8
810398    27
815230    10
819061    70
842246     5
851857    19
854079    52
854548     8
861459     8
871642    54
872180     4
883691    70
890189    10
905061     4
905745    60
914348    70
923841    68
924154    34
948936    13
950264    16
953842    21
961362    70
961962    70
963845    37
973106    12
978089    39
Length: 140, dtype: int64

In [13]:
ttt = df[['sale_date','class_id','sale_quantity']].groupby(['sale_date', 'class_id']).sum()

In [17]:
ttt.unstack().to_csv('../../data/eda/各车型历史销量走势.csv')

In [6]:
cls_in_month = df.groupby('sale_date')['class_id'].count()
cls_in_month

sale_date
2012-01-01    171
2012-02-01    153
2012-03-01    168
2012-04-01    156
2012-05-01    166
2012-06-01    159
2012-07-01    176
2012-08-01    172
2012-09-01    187
2012-10-01    183
2012-11-01    184
2012-12-01    183
2013-01-01    213
2013-02-01    188
2013-03-01    178
2013-04-01    192
2013-05-01    193
2013-06-01    195
2013-07-01    211
2013-08-01    218
2013-09-01    225
2013-10-01    235
2013-11-01    235
2013-12-01    221
2014-01-01    260
2014-02-01    230
2014-03-01    240
2014-04-01    256
2014-05-01    278
2014-06-01    267
             ... 
2015-05-01    352
2015-06-01    350
2015-07-01    349
2015-08-01    350
2015-09-01    375
2015-10-01    370
2015-11-01    383
2015-12-01    384
2016-01-01    378
2016-02-01    356
2016-03-01    369
2016-04-01    337
2016-05-01    300
2016-06-01    304
2016-07-01    305
2016-08-01    312
2016-09-01    331
2016-10-01    354
2016-11-01    368
2016-12-01    387
2017-01-01    369
2017-02-01    351
2017-03-01    384
2017-04-01    373


In [37]:

gd = df.groupby(['class_id','gearbox_type']).sum().sort_values(ascending=False, by=['sale_quantity']).reset_index()
# gd[gd['if_MPV_id']==1]
gd.groupby('class_id').first().reset_index()['gearbox_type']

0       AT
1       AT
2       AT
3       AT
4       MT
5       MT
6       MT
7       AT
8       AT
9      DCT
10      MT
11      AT
12      AT
13      MT
14     DCT
15     DCT
16      AT
17     CVT
18     DCT
19      MT
20      MT
21     DCT
22     DCT
23      AT
24     DCT
25      AT
26      AT
27      MT
28      MT
29     CVT
      ... 
110    DCT
111     AT
112    CVT
113     AT
114    CVT
115     AT
116    DCT
117     AT
118    DCT
119     AT
120     AT
121     AT
122    DCT
123     AT
124    DCT
125    DCT
126     MT
127     MT
128     AT
129     AT
130     AT
131     AT
132     MT
133    DCT
134    DCT
135    CVT
136     MT
137     AT
138    CVT
139     MT
Name: gearbox_type, Length: 140, dtype: object

In [39]:
modes = df[['class_id','brand_id']].groupby('class_id').first().reset_index()
modes

,class_id,brand_id
0,103507,831
1,124140,783
2,125403,761
3,136916,106
4,169673,682
5,175962,750
6,178529,98
7,186250,761
8,194201,831
9,194450,836


In [40]:


for col_name in df.drop(['class_id','sale_date','sale_quantity','brand_id'], axis=1).columns.values:
    gd = df.groupby(['class_id',col_name]).sum().sort_values(ascending=False, by=['sale_quantity']).reset_index()
    modes[col_name] = gd.groupby('class_id').first().reset_index()[col_name]

modes

,class_id,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,...,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,103507,831,2,1,3.0,3,6,AT,1.6,L,...,150.7,4270,1780,1630,1830,1325,5,2590,1557,1570
1,124140,783,2,1,1.0,1,6,AT,1.8,T,...,230.0,4650,1850,1695,2075,1665,5,2700,1565,1565
2,125403,761,2,3,2.0,2,6,AT,2.4,L,...,225.0,5213,1878,1772,2470,1840,7,3088,1593,1601
3,136916,106,3,2,2.0,5,4,AT,2.0,L,...,190.0,4825,1825,1480,2000,1465,5,2775,1575,1560
4,169673,682,2,1,2.0,1,6,MT,1.5,T,...,215.0,4715,1830,1780,2095,1570,7,2760,1547,1550
5,175962,750,3,2,1.0,4,5,MT,1.6,L,...,155.0,4473,1706,1469,1600,1120,5,2603,1460,1500
6,178529,98,1,4,1.0,1,5,MT,1.0,L,...,85.0,3730,1510,1860,1575,985,7,2500,1290,1290
7,186250,761,2,2,3.0,2,6,AT,1.5,L,...,146.0,4370,1809,1461,1715,1280,5,2662,1540,1562
8,194201,831,2,2,1.0,3,6,AT,1.4,L,...,132.4,4240,1750,1505,1570,1145,5,2600,1524,1529
9,194450,836,2,1,1.0,3,7,DCT,1.6,T,...,265.0,4475,1850,1655,2090,1541,5,2670,1620,1631


In [44]:
modes.count()

class_id                 140
brand_id                 140
compartment              140
type_id                  140
level_id                 139
department_id            140
TR                       140
gearbox_type             140
displacement             140
if_charging              140
price_level              140
price                    130
driven_type_id           140
fuel_type_id             140
newenergy_type_id        140
emission_standards_id    140
if_MPV_id                140
if_luxurious_id          140
power                    140
cylinder_number          140
engine_torque            139
car_length               140
car_width                140
car_height               140
total_quality            140
equipment_quality        140
rated_passenger          140
wheelbase                140
front_track              140
rear_track               140
dtype: int64